### Question 2: CRUD Operations in SQL Server
**Objective:** Basic SQL Server interactions.

**Task:**
- Create an SQL table schema to store time-series metal prices. Include fields like `Date`, `Metal`, `Price`.
- Demonstrate basic CRUD operations


In [44]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [45]:
# reading in the data 
data = pd.read_csv(r"C:\Users\lagan\Desktop\OilDesk-Intern-Assessment\data\MarketData.csv")
data.columns = ['Date','LME_Copper_3MO', 'LME_Aluminum_3MO', 'LME_Zinc_3MO','LME_Lead_3MO', 'LME_Tin_3MO', 'Generic_CL_Future']
data = data.drop(index = data.index[:6])

In [46]:
# transforming the data into the functional datatypes
data['Date'] = pd.to_datetime(data['Date'], dayfirst= True)

metal_cols = ['LME_Copper_3MO', 'LME_Aluminum_3MO', 'LME_Zinc_3MO','LME_Lead_3MO', 'LME_Tin_3MO', 'Generic_CL_Future']

for metal in metal_cols:
    data[metal] = pd.to_numeric(data[metal])
    
data.dtypes ## checking its been transformed 

Date                 datetime64[ns]
LME_Copper_3MO              float64
LME_Aluminum_3MO            float64
LME_Zinc_3MO                float64
LME_Lead_3MO                float64
LME_Tin_3MO                   int64
Generic_CL_Future           float64
dtype: object

In [47]:
## setting the Date as the index
data.set_index('Date', inplace=True)
data.head()

,LME_Copper_3MO,LME_Aluminum_3MO,LME_Zinc_3MO,LME_Lead_3MO,LME_Tin_3MO,Generic_CL_Future
Date,,,,,,
2010-01-01,7375.0,2230.0,2560.0,2432.0,16950,79.36
2010-01-04,7500.0,2267.0,2574.0,2515.0,17450,81.51
2010-01-05,7485.0,2302.0,2575.0,2522.5,17375,81.77
2010-01-06,7660.0,2377.0,2718.0,2680.0,17825,83.18
2010-01-07,7535.0,2310.0,2607.0,2599.0,17475,82.66


In [48]:
## connecting to the database
connection = sqlite3.connect('metal_prices.db')
cursor = connection.cursor()

In [49]:
## avoiding issues with database locking
cursor.execute('PRAGMA journal_mode = WAL;')

In [50]:
##  pulling the data from the df to the database - creating a new
## table for each metal 
for metal in data.columns:
    table_name = metal
    
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name}(
    date DATE PRIMARY KEY,
    price REAL)""")
    
    
    
    for date, price in data[metal].items():
        date = date.strftime('%Y-%m-%d')
        cursor.execute(f"""
        INSERT OR REPLACE INTO {table_name}(date,price) VALUES (?,?)
        """ ,(date, price))
        
    connection.commit() ## committing after each table to avoid db locking
connection.close() 

In [51]:
## reconnecting and checking the tables have been created
connection = sqlite3.connect('metal_prices.db')
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()

[('LME_Copper_3MO',),
 ('LME_Aluminum_3MO',),
 ('LME_Zinc_3MO',),
 ('LME_Lead_3MO',),
 ('LME_Tin_3MO',),
 ('Generic_CL_Future',)]

In [52]:
## checking the data has been loaded to the database
cursor.execute("SELECT * FROM 'LME_Copper_3MO'ORDER BY date LIMIT 10").fetchall()

[('2010-01-01', 7375.0),
 ('2010-01-04', 7500.0),
 ('2010-01-05', 7485.0),
 ('2010-01-06', 7660.0),
 ('2010-01-07', 7535.0),
 ('2010-01-08', 7461.0),
 ('2010-01-11', 7567.5),
 ('2010-01-12', 7455.0),
 ('2010-01-13', 7485.0),
 ('2010-01-14', 7490.0)]

In [53]:
## checking prices and dates match
data['LME_Copper_3MO'].head(10)

Date
2010-01-01    7375.0
2010-01-04    7500.0
2010-01-05    7485.0
2010-01-06    7660.0
2010-01-07    7535.0
2010-01-08    7461.0
2010-01-11    7567.5
2010-01-12    7455.0
2010-01-13    7485.0
2010-01-14    7490.0
Name: LME_Copper_3MO, dtype: float64

In [54]:
## query-ing the lowest prices for copper and the dates in they occured
cursor.execute("SELECT * FROM 'LME_Copper_3MO' ORDER BY price ASC LIMIT 10").fetchall()

[('2016-01-15', 4331.0),
 ('2016-01-12', 4352.0),
 ('2016-01-20', 4359.0),
 ('2016-01-18', 4377.0),
 ('2016-01-11', 4387.0),
 ('2016-01-13', 4392.0),
 ('2016-01-19', 4408.0),
 ('2016-01-14', 4415.5),
 ('2016-01-25', 4417.0),
 ('2016-01-21', 4430.0)]

In [55]:
#cRud
## query-ing the average price of the 3m copper during every month 
cursor.execute("""
    SELECT strftime('%Y-%m', date) AS month_year, ROUND(AVG(price),2) as avg_price
    FROM 'LME_Copper_3MO'
    GROUP BY month_year
    ORDER BY month_year;""").fetchall()

[('2010-01', 7392.74),
 ('2010-02', 6891.65),
 ('2010-03', 7501.26),
 ('2010-04', 7774.36),
 ('2010-05', 6904.48),
 ('2010-06', 6530.0),
 ('2010-07', 6772.45),
 ('2010-08', 7331.02),
 ('2010-09', 7739.77),
 ('2010-10', 8302.76),
 ('2010-11', 8440.23),
 ('2010-12', 9127.37),
 ('2011-01', 9520.57),
 ('2011-02', 9872.05),
 ('2011-03', 9514.39),
 ('2011-04', 9514.33),
 ('2011-05', 8979.0),
 ('2011-06', 9077.82),
 ('2011-07', 9670.29),
 ('2011-08', 9020.52),
 ('2011-09', 8321.18),
 ('2011-10', 7407.48),
 ('2011-11', 7602.3),
 ('2011-12', 7584.09),
 ('2012-01', 8052.36),
 ('2012-02', 8450.77),
 ('2012-03', 8455.02),
 ('2012-04', 8230.05),
 ('2012-05', 7854.46),
 ('2012-06', 7423.95),
 ('2012-07', 7583.73),
 ('2012-08', 7523.24),
 ('2012-09', 8094.9),
 ('2012-10', 8063.63),
 ('2012-11', 7721.3),
 ('2012-12', 7972.76),
 ('2013-01', 8078.24),
 ('2013-02', 8094.08),
 ('2013-03', 7676.98),
 ('2013-04', 7264.82),
 ('2013-05', 7278.04),
 ('2013-06', 7032.05),
 ('2013-07', 6916.43),
 ('2013-08', 721

In [56]:
## crUd
## function which updates the price at a certain date

def table_update(table, new_price, date):
    cursor.execute(f"""
        UPDATE {table} SET price = ? WHERE date = ?
    """, (new_price, date))
    connection.commit()
    print(f"Rows updated: {cursor.rowcount}")

    
table = 'Generic_CL_Future'
new_price = 7200
date = '2020-12-31'  
table_update(table,new_price,date)

## checking the price has been updated successfuly 
cursor.execute(f"""SELECT * FROM {table} WHERE date = ?""", (date,)).fetchall()



Rows updated: 1


[('2020-12-31', 7200.0)]

In [57]:
##cruD
## function which deletes a range of dates or else a single date

def table_delete(table, start_date=None, end_date=None):
    if start_date and end_date:
        cursor.execute(f"DELETE FROM {table} WHERE date BETWEEN ? AND ?", (start_date, end_date))
        print(f"Rows deleted between {start_date} and {end_date}")
    elif start_date:
        cursor.execute(f"DELETE FROM {table} WHERE date < ?", (start_date,))
        print(f"Rows deleted before {start_date}")
    else:
        print("No date provided for deletion.")
        return

    connection.commit()
    print(f"Rows deleted: {cursor.rowcount}")

table = 'Generic_CL_Future'
start_date='2011-01-01'

## Delete rows before a specific date
table_delete(table, start_date)

## checking the rows have been deleted
cursor.execute(f"""SELECT * FROM {table} WHERE date < ?""", (start_date,)).fetchall()


Rows deleted before 2011-01-01
Rows deleted: 261


[]

In [58]:
connection.close()